In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

def test_scoring_sanity():
    print("✅ from scoring.ipynb")

def evaluate_predictions(pred_mask, true_mask):
    """
    Compare predicted and true class masks and print evaluation metrics.
    Expects:
    - pred_mask: 2D array of predicted class indices
    - true_mask: 2D array of ground truth class indices
    """
    print("🔎 Evaluating predictions...")

    pred_flat = pred_mask.flatten()
    true_flat = true_mask.flatten()

    if pred_flat.shape != true_flat.shape:
        raise ValueError("Prediction and ground truth shapes don't match.")

    print("📈 Classification Report:")
    print(classification_report(true_flat, pred_flat, labels=np.arange(6), zero_division=0))

    print("\n🌀 Confusion Matrix:")
    cm = confusion_matrix(true_flat, pred_flat, labels=np.arange(6))
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=np.arange(6), yticklabels=np.arange(6))
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.tight_layout()
    plt.show()
